In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

import jb_helper_functions_prep
from jb_helper_functions_prep import create_enc

import prep_telco
from prep_telco import prep_telco_df

from sklearn.model_selection import train_test_split

In [2]:
df = prep_telco_df()
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,onlinesecurity_enc,onlinebackup_enc,deviceprotection_enc,techsupport_enc,streamingtv_enc,streamingmovies_enc,contract_enc,paperlessbilling_enc,paymentmethod_enc,churn_enc
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,0,2,0,0,0,0,0,1,2,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,2,0,2,0,0,0,1,0,3,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,2,2,0,0,0,0,0,1,3,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,2,0,2,2,0,0,1,0,0,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,0,0,0,0,0,0,0,1,2,1


In [3]:
train, test = train_test_split(df, test_size=.3, random_state=123, stratify=df[['churn_enc']])

### A few first thoughts:

### Customers with higher monthly costs will churn more.

### Customers with a longer tenure will churn more and pay more per month.

### Customer with month-to-month contracts will churn more frequently.  They will also pay more per month as we would expect a company to have lower rates for a customer willing to sign a year or two contract.

In [4]:
train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn',
       'gender_enc', 'partner_enc', 'dependents_enc', 'phoneservice_enc',
       'multiplelines_enc', 'internetservice_enc', 'onlinesecurity_enc',
       'onlinebackup_enc', 'deviceprotection_enc', 'techsupport_enc',
       'streamingtv_enc', 'streamingmovies_enc', 'contract_enc',
       'paperlessbilling_enc', 'paymentmethod_enc', 'churn_enc'],
      dtype='object')

In [9]:
pd.DataFrame((train.groupby(['contract', 'churn'])['monthlycharges'].count())/(train.groupby('contract')['monthlycharges'].count()))

monthlycharges
contract       churn                
Month-to-month No           0.573054
               Yes          0.426946
One year       No           0.888454
               Yes          0.111546
Two year       No           0.973639
               Yes          0.026361

### Looking above, we see that 42.7% of month-to-month customers did churn vs. 11.1% of one-year and 2.6% of two-year contract customers.

In [17]:
pd.DataFrame(train.groupby(['internetservice', 'contract'])['monthlycharges'].quantile(.25))

monthlycharges
internetservice contract                      
DSL             Month-to-month         44.4500
                One year               53.7500
                Two year               59.9625
Fiber optic     Month-to-month         76.5500
                One year               92.3000
                Two year               98.9500
No              Month-to-month         19.6000
                One year               19.7000
                Two year               19.7500

In [18]:
pd.DataFrame(train.groupby(['internetservice', 'contract'])['monthlycharges'].quantile(.5))

monthlycharges
internetservice contract                      
DSL             Month-to-month          50.250
                One year                60.950
                Two year                73.100
Fiber optic     Month-to-month          85.700
                One year                99.900
                Two year               105.625
No              Month-to-month          20.050
                One year                20.100
                Two year                20.350

In [19]:
pd.DataFrame(train.groupby(['internetservice', 'contract'])['monthlycharges'].mean())

monthlycharges
internetservice contract                      
DSL             Month-to-month       50.362718
                One year             61.603050
                Two year             70.098190
Fiber optic     Month-to-month       86.804274
                One year             98.655913
                Two year            104.118412
No              Month-to-month       20.307345
                One year             20.883594
                Two year             21.717123

In [15]:
pd.DataFrame(train.groupby(['internetservice', 'contract'])['monthlycharges'].quantile(.75))

monthlycharges
internetservice contract                      
DSL             Month-to-month         57.9500
                One year               71.5000
                Two year               82.4125
Fiber optic     Month-to-month         95.3000
                One year              105.2500
                Two year              111.1125
No              Month-to-month         20.4000
                One year               20.6500
                Two year               24.5875

### In the four tables above, we see that for DSL and Fiber optic, that m-t-m customers are paying less than one and two year contract customers.  This is different than the original thought.  To try and answer this, is it because customers in the m-t-m contracts have a significantly lower tenure so their rates haven't increased yet?  (Think about what happens each year as you're with an internet company.)

### We will engineer a column for years of tenure and take a look at customers and their rates with that approach.